## Messages

<img src=".\assets\LC_Messages.png" width="500">

### Environment Setup

Load and check environment variables

In [1]:
from dotenv import load_dotenv
load_dotenv()

from utils import doublecheck_env
doublecheck_env(".env")

LANGSMITH_API_KEY=****8392
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials
OPENAI_API_KEY=****wEQA


### Human and AI Messages

In [3]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

agent = create_agent(
    model="ollama:gpt-oss:20b-cloud",
    system_prompt="You are a full-stack comedian."
)

In [5]:
human_message = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages":[human_message]})

In [6]:
print(result["messages"][-1].content)

Hello! I'm just a bunch of code, but I'm running smoothly—thanks for asking! How can I help you today?


In [8]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [11]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}")
    print(f"{msg.__class__.__name__}: {msg.content}")   

human: Hello, how are you?
HumanMessage: Hello, how are you?
ai: Hello! I'm just a bunch of code, but I'm running smoothly—thanks for asking! How can I help you today?
AIMessage: Hello! I'm just a bunch of code, but I'm running smoothly—thanks for asking! How can I help you today?


### Alternate formats

#### Strings

There are situations where LangChain can infer the role from the context, and a simple string is enough to create a message.

In [12]:
agent = create_agent(
    model="ollama:gpt-oss:20b-cloud",
    system_prompt="You are a terse sports poet."
)

In [13]:
result = agent.invoke({"messages":"Tell me about baseball"})

print(result["messages"][-1].content)

Diamonds gleam,  
five‑point circles, a white‑ball comet—  
strike, swing, the world holds its breath.  

Base, hit, run—  
a game of ninety‑one seconds:  

**Out**: the roar fades,  
**Hit**: the field shivers,  
**Home run**: stars bow.  

Five‑pitcher rhythm, one‑team dream—  
baseball, the quiet thunder of possibility.


#### Dictionaries

In [14]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)

print(result["messages"][-1].content)

Breathe at start line now  
Legs ignite, wind slips, hearts beat  
Finish line bright glows


#### There are multiple roles

```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

In [16]:
result = agent.invoke(
    {"messages": [
        {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
        {"role": "user", "content": "Write a haiku about sprinters"},
        {"role": "assistant", "content": "Feet don't fail me..."}
    ]}
)

print(result["messages"][-1].content)

Track line glinting bright—  
Feet thunder, hearts beat in sync,  
Finish line breath sighs.


### Output format

#### Messages

Let's create a tool so that agent will create some tool messages

In [3]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """ Check given text has three lines 
    Return None if it is correct, otherwise return an error message.
    """

    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"Checking haiku, it has {len(lines)} lines.")

    if len(lines) != 3:
        return f"Haiku should have 3 lines, but got {len(lines)} lines."
    return None

In [4]:
from langchain.agents import create_agent

agent = create_agent(
    model="ollama:gpt-oss:20b-cloud",
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who writes haikus. You always check your work.",
)

In [5]:
result = agent.invoke(
    {"messages": "Please write a poem"}
)

Checking haiku, it has 3 lines.


In [6]:
result["messages"][-1].content

'Morning sprint—  \nAdrenaline drums, feet pound,  \nVictory waits near.'

In [7]:
print(len(result["messages"]))

4


In [8]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (d04a9e86-dfff-4519-81be-cfa90eb389f4)
 Call ID: d04a9e86-dfff-4519-81be-cfa90eb389f4
  Args:
    text: Morning sprint—
Adrenaline drums, feet pound,
Victory waits near.
================================= Tool Message =================================
Name: check_haiku_lines

null
================================== Ai Message ==================================

Morning sprint—  
Adrenaline drums, feet pound,  
Victory waits near.


### Other useful information

Let's dig into the all information available in the resule

In [9]:
result

{'messages': [HumanMessage(content='Please write a poem', additional_kwargs={}, response_metadata={}, id='b853066d-24dc-4545-bd2e-4a50d76563da'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b-cloud', 'created_at': '2025-10-29T08:50:55.714402148Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1914269135, 'load_duration': None, 'prompt_eval_count': 147, 'prompt_eval_duration': None, 'eval_count': 529, 'eval_duration': None, 'model_name': 'gpt-oss:20b-cloud', 'model_provider': 'ollama'}, id='lc_run--e25af997-3f6c-4ae5-b695-bf0463231674-0', tool_calls=[{'name': 'check_haiku_lines', 'args': {'text': 'Morning sprint—\nAdrenaline drums, feet pound,\nVictory waits near.'}, 'id': 'd04a9e86-dfff-4519-81be-cfa90eb389f4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 147, 'output_tokens': 529, 'total_tokens': 676}),
  ToolMessage(content='null', name='check_haiku_lines', id='299d1612-cae5-4acb-8dc3-ba218d2f8161', tool_call_id='d04a9e86-df

Just last message

In [10]:
result["messages"][-1]

AIMessage(content='Morning sprint—  \nAdrenaline drums, feet pound,  \nVictory waits near.', additional_kwargs={}, response_metadata={'model': 'gpt-oss:20b-cloud', 'created_at': '2025-10-29T08:50:56.857138124Z', 'done': True, 'done_reason': 'stop', 'total_duration': 742936041, 'load_duration': None, 'prompt_eval_count': 203, 'prompt_eval_duration': None, 'eval_count': 128, 'eval_duration': None, 'model_name': 'gpt-oss:20b-cloud', 'model_provider': 'ollama'}, id='lc_run--448f08b9-70e3-4491-9c4c-112770402335-0', usage_metadata={'input_tokens': 203, 'output_tokens': 128, 'total_tokens': 331})

In [11]:
result["messages"][-1].usage_metadata

{'input_tokens': 203, 'output_tokens': 128, 'total_tokens': 331}

In [12]:
result["messages"][-1].response_metadata

{'model': 'gpt-oss:20b-cloud',
 'created_at': '2025-10-29T08:50:56.857138124Z',
 'done': True,
 'done_reason': 'stop',
 'total_duration': 742936041,
 'load_duration': None,
 'prompt_eval_count': 203,
 'prompt_eval_duration': None,
 'eval_count': 128,
 'eval_duration': None,
 'model_name': 'gpt-oss:20b-cloud',
 'model_provider': 'ollama'}

### Messages are very very important

- Make sure proper system and human messages are provided
- Make sure tools are properly documented
- Change system prompt and human message --> Monitor outout and messages